# Elements logiciels pour le traitement des données en grande dimension

## Installation de pyspark

In [1]:
import findspark
sparkHome = "C:\Dev\Spark\spark-2.2.0-bin-hadoop2.7"
findspark.init(sparkHome)

### Petit essai pour calculer PI

In [2]:
def printSpendTime (startTime) :
    spendTime = time.time() - startTime
    m, s = divmod(spendTime, 60)
    h, m = divmod(m, 60)
    print("--------------------------------------------------")
    print ("-----   Temps écoulé : %dh%02dm%02ds" % (h, m, s))
    print("--------------------------------------------------")
    return

In [3]:
import pyspark
import random
import time

startTime = time.time()

sc = pyspark.SparkContext(appName="Pi")
num_samples = 100000000

def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()

pi = 4 * count / num_samples
print(pi)

sc.stop()

printSpendTime(startTime)

3.14170012
--------------------------------------------------
-----   Temps écoulé : 0h00m27s
--------------------------------------------------


## Arbre k-d classique

### Création d'un jeu de données

In [4]:
import numpy as np
import pandas

n = 10
data = pandas.DataFrame(np.random.rand(n,2)*100,  columns = ['X', 'Y'])
data

,X,Y
0,12.029079,61.682784
1,91.984005,23.328280
2,50.283671,38.211646
3,57.839173,21.941922
4,10.536263,22.992834
5,51.427709,24.904803
6,43.532735,17.592368
7,16.898368,92.266527
8,5.462290,72.978728
9,88.935303,39.293454


In [5]:
data.index.values.tolist()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]